In [1]:
import pandas as pd
import tables as tb
import tstables
from datetime import datetime, timedelta


class SymbolTableDescription(tb.IsDescription):
    timestamp = tb.Int64Col(pos = 0)
    open = tb.Float64Col(pos = 1)
    high = tb.Float64Col(pos = 2)
    low = tb.Float64Col(pos = 3)
    close = tb.Float64Col(pos = 4)
    volume = tb.Float64Col(pos = 5)


# BTCUSDT-1m-2023-02-20.csv


In [2]:
csv_file_path = 'store/BTCUSDT-1m-2023-02-20.csv'

data = pd.read_csv(csv_file_path)
data.drop(columns = ['close_time', 'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume', 'ignore'], inplace = True)
data

,open_time,open,high,low,close,volume
0,1676851200000,24265.9,24309.9,24263.5,24309.5,910.779
1,1676851260000,24309.6,24314.9,24283.1,24314.8,864.957
2,1676851320000,24314.7,24355.7,24305.7,24348.3,1312.078
3,1676851380000,24348.3,24369.4,24345.7,24358.7,871.238
4,1676851440000,24358.8,24404.1,24358.7,24395.8,1342.734
...,...,...,...,...,...,...
1435,1676937300000,24813.5,24826.4,24810.7,24826.3,252.998
1436,1676937360000,24826.2,24847.4,24826.2,24838.6,543.747
1437,1676937420000,24838.7,24841.0,24833.2,24838.9,169.390
1438,1676937480000,24838.8,24852.9,24838.0,24852.5,208.190


In [3]:
data['open_time'] = pd.to_datetime(data['open_time'], unit = 'ms')
data['open'] = pd.to_numeric(data['open'])
data['high'] = pd.to_numeric(data['high'])
data['low'] = pd.to_numeric(data['low'])
data['close'] = pd.to_numeric(data['close'])
data['volume'] = pd.to_numeric(data['volume'])
data.set_index('open_time', inplace = True)
data

,open,high,low,close,volume
open_time,,,,,
2023-02-20 00:00:00,24265.9,24309.9,24263.5,24309.5,910.779
2023-02-20 00:01:00,24309.6,24314.9,24283.1,24314.8,864.957
2023-02-20 00:02:00,24314.7,24355.7,24305.7,24348.3,1312.078
2023-02-20 00:03:00,24348.3,24369.4,24345.7,24358.7,871.238
2023-02-20 00:04:00,24358.8,24404.1,24358.7,24395.8,1342.734
...,...,...,...,...,...
2023-02-20 23:55:00,24813.5,24826.4,24810.7,24826.3,252.998
2023-02-20 23:56:00,24826.2,24847.4,24826.2,24838.6,543.747
2023-02-20 23:57:00,24838.7,24841.0,24833.2,24838.9,169.390


In [4]:
symbol = 'BTCUSDT'
store_path = f'store\{symbol}.h5'
store = tb.open_file(store_path, 'a')
table = None

table = store.create_ts('/', symbol, SymbolTableDescription)
table.append(data)

sdate = datetime.now() - timedelta(days=5)
edate = datetime.now()
rows = table.read_range(sdate, edate)
rows

,open,high,low,close,volume
2023-02-20 11:17:00,24886.6,24886.6,24868.2,24871.9,247.454
2023-02-20 11:18:00,24871.9,24873.8,24862.6,24862.6,183.179
2023-02-20 11:19:00,24862.7,24873.9,24860.1,24872.6,234.547
2023-02-20 11:20:00,24872.6,24901.2,24872.1,24884.1,371.009
2023-02-20 11:21:00,24884.2,24900.0,24874.2,24900.0,213.697
...,...,...,...,...,...
2023-02-20 23:55:00,24813.5,24826.4,24810.7,24826.3,252.998
2023-02-20 23:56:00,24826.2,24847.4,24826.2,24838.6,543.747
2023-02-20 23:57:00,24838.7,24841.0,24833.2,24838.9,169.390
2023-02-20 23:58:00,24838.8,24852.9,24838.0,24852.5,208.190


In [5]:
store.close()